## LangChain 출력 파서 (Output Parser)
- LangChain 출력 파서는 LLM이 생성한 텍스트를 원하는 형식(문자열, 리스트, JSON 등)으로 변환하거나 검증하는 도구이다.<br>
  자유 텍스트는 StrOutputParser, 구조화된 데이터는 PydanticOutputParser처럼 목적에 맞게 선택해 사용한다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] StrOutputParser
 : LLM 출력 텍스트를 그대로 반환하는 가장 단순한 파서

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1) 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 2) 프롬프트 템플릿 작성
prompt = ChatPromptTemplate.from_template("다음 주제로 간단한 시를 작성해줘 : {topic}.")

# 3) 출력 파서 (문자열 그대로)
parser = StrOutputParser()

# 4) 체인 구성: 프롬프트 → LLM → 문자열 파싱
chain = prompt | llm | parser

# 5) 실행
result = chain.invoke({"topic": "바다"})
print(result)

푸른 물결 속에 숨겨진 꿈,  
파도는 속삭이고, 바람은 춤춘다.  
해가 지고, 노을이 물들면,  
바다는 수천 가지 이야기를 품는다.  

모래사장 위 발자국,  
그리움과 희망을 담고,  
끝없이 펼쳐진 수평선,  
자유를 찾아 떠나는 항해.  

조개 껍데기, 별빛처럼 빛나고,  
파란 심연 속에 비밀이 숨어,  
바다여, 너는 나의 안식처,  
영원히 나의 마음을 감싸 안아줘.  


In [3]:
type(result)  # str : StrOutputParser는 출력이 문자열(str)

str

In [6]:
# type(parser)
# dir(parser)  # 'invoke','parse'

langchain_core.output_parsers.string.StrOutputParser

In [9]:
ret = parser.invoke("하늘이 파란이유는?")
print(ret,type(ret))

하늘이 파란이유는? <class 'str'>


In [11]:
ret = parser.parse("하늘이 파란이유는?")
print(ret,type(ret))

하늘이 파란이유는? <class 'str'>


### [2] PydanticOutputParser

#### Pydantic이란?

: Pydantic은 Python에서 데이터 검증과 설정 관리를 쉽게 할 수 있도록 도와주는 데이터 검증 라이브러리이다. <br>
Python의 타입 힌트(type hint) 를 기반으로 동작한다. <br>
입력 데이터를 검사하고, 올바른 타입으로 자동 변환해 준다. <br>
잘못된 데이터가 들어오면 ValidationError 를 발생시켜 안전성을 높인다. <br>
한마디로 “데이터를 정해진 스키마(schema)대로 안전하게 다루게 해주는 도구”이다. <br>

### LangChain의 PydanticOutputParser는
: LLM이 출력한 텍스트를 Pydantic 모델로 변환해 주는 출력 파서이다. <br>

- 특징
LLM이 만든 JSON 같은 문자열을 자동으로 Python 객체로 바꿔준다. <br>
유효성 검사(validation) 기능이 내장되어 있어, 누락된 필드나 잘못된 데이터 타입을 잡아낼 수 있다. <br>
LLM이 JSON 형식을 틀리게 출력했을 때도, 자동으로 재시도하거나 오류를 알려줄 수 있다. 

In [28]:
from pydantic import BaseModel

# 데이터 스키마 정의
class User(BaseModel):
    id: int
    name: str
    age: int

# 올바른 데이터 → 정상 객체 생성
user = User(id=1, name="홍길동", age=25)
print(user)  
# User(id=1, name='홍길동', age=25)

# 잘못된 데이터 → 자동 변환 또는 에러
user2 = User(id="2", name="이몽룡", age="30")
print(user2)
# User(id=2, name='이몽룡', age=30)   ← 문자열이 int로 자동 변환됨

# 타입 불일치 → 에러 발생
# user3 = User(id="삼", name="성춘향", age="스무살")
# pydantic.error_wrappers.ValidationError 발생

id=1 name='홍길동' age=25
id=2 name='이몽룡' age=30


In [29]:
# 형변환
s = "2"
a = int(s)
print(a,type(a))

s = "2.0"
a = float(s)
print(a,type(a))


2 <class 'int'>
2.0 <class 'float'>


In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

# 1) 출력 데이터 구조 정의 (Pydantic 모델)
class Joke(BaseModel):
    setup: str = Field(description="농담의 전개 부분")
    punchline: str = Field(description="농담의 결론/웃긴 부분")

# 2) 파서 생성
parser = PydanticOutputParser(pydantic_object=Joke)



In [36]:
# dir(BaseModel)

In [34]:
dir(Field)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']